# AdaGrad Optimizer from Scratch

This notebook implements the AdaGrad (Adaptive Gradient) optimizer from scratch using only NumPy, including support for learning rate, numerical stability (eps), and optional weight decay (L2 regularization).


## Imports


In [ ]:
import numpy as np

print("Libraries imported successfully!")


## AdaGrad Optimizer Implementation


In [ ]:
class AdaGrad:
    """
    AdaGrad (Adaptive Gradient) optimizer implemented from scratch using NumPy.
    
    AdaGrad adapts the learning rate for each parameter by accumulating the squared gradients
    throughout training. Parameters with large accumulated squared gradients will have smaller
    learning rates, while parameters with small accumulated squared gradients will have relatively
    larger learning rates.
    
    Parameters:
    -----------
    lr : float, optional
        Learning rate (step size) for weight updates (default: 0.01)
    eps : float, optional
        Small constant added to the denominator for numerical stability (default: 1e-8)
    weight_decay : float, optional
        L2 regularization coefficient (default: 0.0)
    
    Algorithm:
    ----------
    1. Accumulate squared gradients: G_t = G_{t-1} + g_t^2
    2. Update parameters: w_t = w_{t-1} - lr * g_t / (√G_t + ε)
    
    Key Characteristics:
    --------------------
    - Accumulates squared gradients from the beginning of training (no decay)
    - Learning rates decrease monotonically over time
    - Works well for sparse gradients and convex problems
    - Can lead to vanishing learning rates in the long run
    
    Examples:
    --------
    >>> optimizer = AdaGrad(lr=0.01, eps=1e-8, weight_decay=0.01)
    >>> optimizer.step(weights, gradients)
    """
    
    def __init__(self, lr=0.01, eps=1e-8, weight_decay=0.0):
        self.lr = lr
        self.eps = eps
        self.weight_decay = weight_decay
        
        # Dictionary to store accumulated squared gradients for each parameter set
        self.G = {}
    
    def step(self, params, grads):
        """
        Perform a single AdaGrad optimization step.
        
        Parameters:
        -----------
        params : numpy.ndarray
            Model parameters (weights) to be updated (modified in-place)
        grads : numpy.ndarray
            Gradients with respect to the parameters
        
        Algorithm:
        ----------
        1. Apply weight decay if enabled: g_t = g_t + weight_decay * w_t
        2. Accumulate squared gradients: G_t = G_{t-1} + g_t^2
        3. Update parameters: w_t = w_{t-1} - lr * g_t / (√G_t + ε)
        """
        # Get unique identifier for this parameter set
        param_id = id(params)
        
        # Initialize accumulated squared gradients if not exists
        if param_id not in self.G:
            self.G[param_id] = np.zeros_like(params)
        
        # Apply L2 regularization (weight decay) to gradients
        if self.weight_decay > 0:
            grads = grads + self.weight_decay * params
        
        # Accumulate squared gradients
        # G_t = G_{t-1} + g_t^2
        self.G[param_id] = self.G[param_id] + (grads ** 2)
        
        # Update parameters
        # w_t = w_{t-1} - lr * g_t / (√G_t + ε)
        params[:] = params - self.lr * grads / (np.sqrt(self.G[param_id]) + self.eps)


## Numerical Example: Parameter Updates Over Several Steps


In [ ]:
# Simple numerical example
import numpy as np

np.random.seed(42)
W = np.random.randn(2, 2)
grads = np.random.randn(2, 2)

print("Initial weights W:")
print(W)
print("\nInitial gradients:")
print(grads)
print("\n" + "="*60 + "\n")

# Create AdaGrad optimizer
adagrad = AdaGrad(lr=0.01, eps=1e-8)

print("AdaGrad optimizer parameters:")
print(f"  Learning rate: {adagrad.lr}")
print(f"  Epsilon: {adagrad.eps}")
print("\n" + "="*60 + "\n")

# Perform 5 optimization steps
for i in range(5):
    adagrad.step(W, grads)
    print(f"Step {i+1}, Updated Weights:\n{W}\n")


## Detailed Example: Showing Intermediate Values (G_t and Adaptive Learning Rates)


In [ ]:
# Detailed example showing intermediate values (G_t and adaptive step sizes)
np.random.seed(42)
W = np.random.randn(2, 2)
grads = np.random.randn(2, 2)

print("Initial weights W:")
print(W)
print("\nInitial gradients:")
print(grads)
print("\n" + "="*70 + "\n")

adagrad = AdaGrad(lr=0.01, eps=1e-8)

param_id = id(W)

for i in range(5):
    print(f"Step {i+1}:")
    print(f"  Learning rate: {adagrad.lr}")
    print(f"  Eps: {adagrad.eps}")
    
    # Store previous values for display
    G_prev = adagrad.G.get(param_id, np.zeros_like(W)).copy()
    W_prev = W.copy()
    
    print(f"\n  Before step():")
    print(f"    G_{i} (accumulated squared gradients): {G_prev}")
    print(f"    W_{i}: {W_prev}")
    print(f"    Current gradients: {grads}")
    print(f"    Gradient magnitudes: {np.abs(grads)}")
    
    # Perform step
    adagrad.step(W, grads)
    
    # Display intermediate values
    print(f"\n  After step() - Updated accumulated squared gradients:")
    print(f"    G_{i+1} = G_{i} + g_{i+1}^2")
    print(f"    G_{i+1} = {G_prev} + {grads**2}")
    print(f"    G_{i+1}: {adagrad.G[param_id]}")
    
    # Calculate adaptive step sizes
    sqrt_G = np.sqrt(adagrad.G[param_id])
    denominator = sqrt_G + adagrad.eps
    adaptive_lr = adagrad.lr / denominator
    step_size = adaptive_lr * grads
    
    print(f"\n  Adaptive step size calculation:")
    print(f"    sqrt(G_{i+1}): {sqrt_G}")
    print(f"    sqrt(G_{i+1}) + eps: {denominator}")
    print(f"    Adaptive learning rate per parameter: {adagrad.lr} / {denominator}")
    print(f"    Adaptive learning rate: {adaptive_lr}")
    print(f"    Step size: lr * g / (sqrt(G) + eps) = {step_size}")
    
    print(f"\n  Parameter update:")
    print(f"    W_{i+1} = W_{i} - lr * g_{i+1} / (sqrt(G_{i+1}) + eps)")
    print(f"    Updated W_{i+1}: {W}")
    print(f"    Change in W: {W - W_prev}")
    print("-" * 70)


## Example: Demonstrating Decreasing Learning Rates Over Time


In [ ]:
# Demonstrate how AdaGrad's learning rate decreases over time
np.random.seed(42)

W = np.random.randn(2, 2)
grads = np.random.randn(2, 2)

print("Demonstrating decreasing learning rates:")
print(f"Initial weights W:\n{W}")
print(f"\nGradients (constant for demonstration):\n{grads}")
print("="*70 + "\n")

adagrad = AdaGrad(lr=0.1, eps=1e-8)  # Use larger lr to see the effect

for i in range(5):
    G_before = adagrad.G.get(id(W), np.zeros_like(W)).copy()
    W_before = W.copy()
    
    # Perform step
    adagrad.step(W, grads)
    
    # Calculate adaptive learning rates
    sqrt_G = np.sqrt(adagrad.G[id(W)])
    adaptive_lr = adagrad.lr / (sqrt_G + adagrad.eps)
    step_size = adaptive_lr * grads
    
    print(f"Step {i+1}:")
    print(f"  G (accumulated squared gradients):\n{adagrad.G[id(W)]}")
    print(f"  sqrt(G):\n{sqrt_G}")
    print(f"  Adaptive learning rate per parameter:\n{adaptive_lr}")
    print(f"  Step size per parameter:\n{step_size}")
    print(f"  Updated W:\n{W}")
    print(f"  Change in W:\n{W - W_before}")
    print(f"\n  Key observation: Learning rate decreases as G accumulates!")
    print(f"  Average adaptive learning rate: {np.mean(adaptive_lr):.6f}")
    print("-" * 70)


## Example: AdaGrad with Weight Decay


In [ ]:
# Example demonstrating AdaGrad with weight decay
np.random.seed(42)

W = np.random.randn(2, 2)
grads = np.random.randn(2, 2)

print("AdaGrad without weight decay:")
adagrad_no_decay = AdaGrad(lr=0.01, eps=1e-8, weight_decay=0.0)
W_no_decay = W.copy()

for i in range(3):
    adagrad_no_decay.step(W_no_decay, grads)
    print(f"Step {i+1}, W:\n{W_no_decay}\n")

print("="*70 + "\n")
print("AdaGrad with weight decay (weight_decay=0.01):")
adagrad_with_decay = AdaGrad(lr=0.01, eps=1e-8, weight_decay=0.01)
W_with_decay = W.copy()

for i in range(3):
    adagrad_with_decay.step(W_with_decay, grads)
    print(f"Step {i+1}, W:\n{W_with_decay}\n")


## Example: Comparing AdaGrad with Different Learning Rates


In [ ]:
# Compare AdaGrad with different learning rates
np.random.seed(42)

W_1 = np.random.randn(2, 2)
W_2 = np.random.randn(2, 2)
grads = np.random.randn(2, 2)

print("AdaGrad with lr=0.01:")
adagrad1 = AdaGrad(lr=0.01, eps=1e-8)
for i in range(3):
    adagrad1.step(W_1, grads)
    print(f"Step {i+1}: {W_1}")

print("\n" + "="*70 + "\n")
print("AdaGrad with lr=0.1:")
adagrad2 = AdaGrad(lr=0.1, eps=1e-8)
for i in range(3):
    adagrad2.step(W_2, grads)
    print(f"Step {i+1}: {W_2}")


## Step-by-Step Explanation

### AdaGrad Algorithm Overview

AdaGrad (Adaptive Gradient) is an adaptive learning rate optimization algorithm that adapts the learning rate for each parameter by accumulating squared gradients throughout training. Unlike RMSProp, AdaGrad accumulates gradients from the beginning without decay.

### Key Components:

1. **Accumulated Squared Gradients**:
   - $G_t = G_{t-1} + g_t^2$
   - Accumulates squared gradients from the start of training (no decay factor)
   - Tracks the total magnitude of gradients seen so far

2. **Parameter Update**:
   - $\theta_t = \theta_{t-1} - \alpha \frac{g_t}{\sqrt{G_t} + \epsilon}$
   - Where $\alpha$ is the learning rate and $\epsilon$ is a small constant for numerical stability
   - Divides gradients by the square root of accumulated squared gradients

3. **Weight Decay (L2 Regularization)**: If enabled, adds penalty to gradients:
   - $g_t = g_t + \text{weight\_decay} \times \theta_t$

### Advantages:

- **Automatic Learning Rate Tuning**: No manual tuning needed for each parameter
- **Works Well with Sparse Gradients**: Effective for sparse data where gradients are zero most of the time
- **Good for Convex Problems**: Performs well on convex optimization problems
- **Smaller Steps for Frequent Features**: Parameters that receive large gradients frequently get smaller learning rates

### Limitations:

- **Monotonically Decreasing Learning Rate**: As $G_t$ accumulates, the learning rate keeps decreasing
- **Vanishing Learning Rate Problem**: In the long run, learning rates can become too small to make meaningful progress
- **Not Ideal for Non-Convex Problems**: Later variants (RMSProp, Adam) perform better for deep learning

### Key Insight:

The algorithm divides the gradient by the square root of accumulated squared gradients. This means:
- Parameters with consistently large gradients will have large $G_t$, resulting in smaller step sizes
- Parameters with small or infrequent gradients will have smaller $G_t$, allowing relatively larger step sizes
- Over time, the learning rate decreases for all parameters, which can lead to premature convergence


## Visual Comparison: AdaGrad vs RMSProp Behavior

Note: This cell demonstrates the conceptual difference between AdaGrad (accumulates forever) 
and RMSProp (exponential moving average with decay). AdaGrad's G keeps growing, while 
RMSProp's v stabilizes due to the decay factor.


In [ ]:
# Demonstrate the key difference: AdaGrad accumulates forever (no decay)
np.random.seed(42)

W = np.random.randn(2, 2)
grads = np.random.randn(2, 2)

print("AdaGrad - Accumulated squared gradients keep growing:")
print("="*70)
adagrad = AdaGrad(lr=0.1, eps=1e-8)

for i in range(5):
    adagrad.step(W, grads)
    G_mean = np.mean(adagrad.G[id(W)])
    sqrt_G_mean = np.mean(np.sqrt(adagrad.G[id(W)]))
    adaptive_lr_mean = np.mean(adagrad.lr / (np.sqrt(adagrad.G[id(W)]) + adagrad.eps))
    print(f"  Step {i+1}: Mean(G) = {G_mean:.6f}, Mean(√G) = {sqrt_G_mean:.6f}, Mean(adaptive_lr) = {adaptive_lr_mean:.6f}")

print("\nKey observation: G accumulates continuously, causing adaptive learning rate to decrease over time.")
print("This is why RMSProp (with decay) was developed to address AdaGrad's vanishing learning rate problem.")
